In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
import tqdm
from tqdm import tqdm
import multiprocessing as mp
import seaborn as sns
from tqdm import trange



sns.set()

%load_ext autoreload
%autoreload 2

In [2]:
# data = pd.read_csv("../data/asset_data_2010to2023_full.csv")
data = pd.read_csv("../data/1970_unclean.csv")

In [3]:
prices = pd.pivot_table(data, values='PRC', index='date', columns='PERMCO', fill_value=None)#.loc['2015-01-01':]
shares_outstanding = pd.pivot_table(data, values='SHROUT', index='date', columns='PERMCO', fill_value=None)#.loc['2015-01-01':]

In [5]:
prices.shape

(2526, 7145)

In [6]:
market_cap = prices.iloc[0]*shares_outstanding.iloc[0]

# Get index of 500 largest market cap stocks
top_500 = market_cap.sort_values(ascending=False).iloc[:500].index

In [7]:
prices_500 = prices[top_500]
returns_500 = prices_500.pct_change()
# Cut returns at +/- 15 %
returns_500 = returns_500.clip(-0.15, 0.15)
# BAckpropogate prices from returns starting at last day
return_cum_prod = returns_500.fillna(0).add(1).cumprod()
prices_500_cleaned = return_cum_prod.multiply(prices_500.iloc[0])

prices_500_cleaned = prices_500_cleaned.where(prices_500.notna())

In [25]:
prices_500_cleaned.to_csv("../data/prices_1970_cleaned.csv")

In [24]:
diff = prices_500 - prices_500_cleaned
(diff.abs() > 1e-10).sum().sum()

355226

In [108]:
(prices_500.pct_change().abs() > 0.15).sum().sum() / (prices_500.shape[0]*prices_500.shape[1])

0.0016256206554121151

1007000